# Simulation Monte Carlo — montée en charge (production) vs demande (marché + clients)

Ce notebook met à jour le modèle en intégrant **explicitement** :
1) la **demande marché** (proxy = importations historiques),
2) une **adoption progressive** par un nouvel entrant via une **part de marché atteignable** $s(t)$,
3) la **demande générée par clients** (processus stochastique),
4) la **demande réelle** comme minimum des contraintes :

$$\text{Ventes}(t)=\min\big(\text{Production dispo}(t),\; \text{Demande clients}(t),\; \text{Demande max adressable}(t)\big)$$

Objectif : obtenir une **courbe d'avancement** (P10/P50/P90) et estimer quand la production vendue atteint **11 000 t/an** puis **15 000 t/an**.


## 0) Plan de mise à jour du notebook
### Étape A — Données marché (exogène)
- Entrer les importations annuelles 2019–2024 (proxy de la demande non couverte localement).
- Estimer : niveau (moyenne), croissance (CAGR), volatilité (écart-type relatif).
- Construire une trajectoire **mensuelle** de demande marché $D_{m}(t)$ avec incertitude (scénarios + bruit).

### Étape B — Adoption nouvel entrant (borne supérieure)
- Définir $s(t)$ = part de marché **atteignable** (0 au départ, montée en S).
- Définir la demande max adressable :
$$D_{addr}(t)=s(t)\,D_{m}(t)$$
- Justification : un nouvel entrant ne capte pas rapidement une part élevée (qualification, multi-sourcing, inertie contrats, crédibilité).

### Étape C — Demande clients (endogène, stochastique)
- Acquisition clients : Poisson avec intensité croissante.
- Onboarding : délai entre lead et commandes récurrentes.
- Volume/client : lognormal (positif, asymétrique).
- Croissance + churn : dynamique mensuelle.

### Étape D — Capacité & production disponible
- Capacité effective = nominale × utilisation cible.
- Ramp-up capacité : logistique.
- Downtime : bruit mensuel (arrêts imprévus).

### Étape E — Monte Carlo + lecture décisionnelle
- Simuler N trajectoires (ex. 2 000).
- Extraire P10/P50/P90 pour capacité, demande, ventes.
- Calculer le premier mois où P50(ventes) franchit 11k et 15k t/an.


In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

rng = np.random.default_rng(42)


## 1) Données marché — importations (proxy de la demande)
On considère les importations comme un **proxy** de la demande adressable localement avant substitution par production locale. Ce n'est pas "ta demande" au démarrage : cela sert à **borner** le maximum vendable via $D_{addr}(t)$.

In [ ]:
imports = pd.Series(
    [16657, 22582, 22711, 23049, 21314, 23853],
    index=pd.Index([2019, 2020, 2021, 2022, 2023, 2024], name="Year"),
    name="Imports_tons"
)
imports


In [ ]:
mean_imports = imports.mean()
std_imports = imports.std(ddof=1)
cv_imports = std_imports / mean_imports
cagr = (imports.loc[2024] / imports.loc[2019]) ** (1/5) - 1

summary_market = pd.DataFrame({
    "Mean (t/an)": [mean_imports],
    "Std (t/an)": [std_imports],
    "CV": [cv_imports],
    "CAGR 2019-2024": [cagr],
})
summary_market


## 2) Construction d'une demande marché mensuelle $D_m(t)$
On projette la demande marché à partir de 2024 sur un horizon mensuel, en combinant :
- une croissance moyenne (scénarios prudent/central/ambitieux),
- une volatilité (bruit) cohérente avec le CV historique.

⚠️ Cette demande marché est **exogène** : elle ne dépend pas de l'usine ni des clients signés, elle décrit la taille du gâteau.

In [ ]:
months = 60
t = np.arange(months + 1)

D0 = float(imports.loc[2024])  # t/an

growth_scenarios = {
    "prudent": 0.03,
    "central": float(cagr),
    "ambitieux": 0.08
}

sigma_y = float(cv_imports)
sigma_m = sigma_y / math.sqrt(12)

def market_demand_path(growth_annual, noise=True, rng_local=None):
    if rng_local is None:
        rng_local = rng
    g_m = (1 + growth_annual) ** (1/12) - 1
    D = np.zeros_like(t, dtype=float)
    D[0] = D0
    for m in range(1, months + 1):
        eps = rng_local.normal(0.0, sigma_m) if noise else 0.0
        D[m] = max(0.0, D[m-1] * (1 + g_m) * (1 + eps))
    return D  # t/an

Dm_demo = {k: market_demand_path(v, noise=False) for k, v in growth_scenarios.items()}


In [ ]:
plt.figure(figsize=(10,5))
for name, Dm in Dm_demo.items():
    plt.plot(t, Dm, label=f"Marché {name}")
plt.xlabel("Temps (mois)")
plt.ylabel("Demande marché (t/an)")
plt.title("Projection exogène de la demande marché (sans bruit)")
plt.legend()
plt.tight_layout()
plt.show()


## 3) Adoption nouvel entrant : part de marché atteignable $s(t)$
On modélise $s(t)$ par une logistique :
$$s(t)=s_{max}\cdot\frac{1}{1+e^{-k_s(t-t_{0,s})}}$$
- $s_{max}$ : plafond de pénétration crédible à moyen terme.
- $t_{0,s}$ : moment où la crédibilité devient forte.
- $k_s$ : vitesse d'adoption.


In [ ]:
def logistic(x, k, x0):
    return 1.0 / (1.0 + np.exp(-k * (x - x0)))

s_max = 0.60
t0_s = 36.0
k_s  = 0.12

s_t = s_max * logistic(t, k_s, t0_s)

plt.figure(figsize=(10,4))
plt.plot(t, 100*s_t)
plt.xlabel("Temps (mois)")
plt.ylabel("s(t) (%)")
plt.title("Part de marché atteignable — s(t)")
plt.tight_layout()
plt.show()


### Demande max adressable
$$D_{addr}(t)=s(t)\,D_m(t)$$
Borne supérieure de ce que le marché est prêt à t'acheter à l'instant t.

In [ ]:
Daddr_demo = s_t * Dm_demo["central"]

plt.figure(figsize=(10,5))
plt.plot(t, Dm_demo["central"], label="Marché (central)")
plt.plot(t, Daddr_demo, label="Max adressable")
plt.xlabel("Temps (mois)")
plt.ylabel("t/an")
plt.title("Marché vs demande max adressable")
plt.legend()
plt.tight_layout()
plt.show()


## 4) Capacité de production (rampe industrielle)
- Capacité effective : $Q_{eff,max}=Q_{max}\times u$
- Ramp-up : $Q(t)=Q_{eff,max}\cdot\frac{1}{1+e^{-k_c(t-t_{0,c})}}$
- Production mensuelle dispo : $P(t)=Q(t)/12\cdot(1-\delta_t)$

In [ ]:
Q_max = 15000.0
u_target = 0.85
Q_eff_max = Q_max * u_target

t0_cap = 22.0
k_cap = 0.22

Q_t = Q_eff_max * logistic(t, k_cap, t0_cap)   # t/an
P_tpm_base = Q_t / 12.0                        # t/mois

downtime_mean = 0.03
downtime_sd   = 0.02

plt.figure(figsize=(10,4))
plt.plot(t, Q_t)
plt.xlabel("Temps (mois)")
plt.ylabel("Capacité effective (t/an)")
plt.title("Ramp-up capacité")
plt.tight_layout()
plt.show()


## 5) Demande générée par clients (stochastique)
- Acquisition : Poisson($\lambda(t)$) avec $\lambda(t)$ croissant
- Onboarding : délai en mois
- Volume/client : lognormal
- Croissance + churn

In [ ]:
lam0 = 0.10
lam_max = 1.20
t0_lam = 18.0
k_lam  = 0.18

onboard_mean = 4.0
onboard_sd = 1.5

mean_target_tpm = 18.0
sigma_logn = 0.6
mu_logn = math.log(mean_target_tpm) - 0.5 * sigma_logn**2

cust_growth_mean = 0.015
cust_growth_sd = 0.010
churn_monthly = 0.01


## 6) Simulation complète : une trajectoire
$$\text{Ventes}(t)=\min\big(P(t),\;D_{clients}(t),\;D_{addr}(t)/12\big)$$

In [ ]:
def simulate_one_path(growth_annual, seed=None):
    rng_local = np.random.default_rng(seed) if seed is not None else rng

    Dm = market_demand_path(growth_annual, noise=True, rng_local=rng_local)  # t/an
    s = s_max * logistic(t, k_s, t0_s)
    Daddr = s * Dm  # t/an

    customers = []
    demand_clients_tpm = np.zeros_like(t, dtype=float)
    prod_tpm = np.zeros_like(t, dtype=float)
    sales_tpm = np.zeros_like(t, dtype=float)

    for m in range(1, months + 1):
        lam_m = lam0 + (lam_max - lam0) * logistic(m, k_lam, t0_lam)
        n_new = rng_local.poisson(lam_m)
        for _ in range(n_new):
            delay = max(0, int(round(rng_local.normal(onboard_mean, onboard_sd))))
            start = min(months, m + delay)
            base = rng_local.lognormal(mu_logn, sigma_logn)
            customers.append({"start": start, "demand": base, "active": True})

        total_demand = 0.0
        for c in customers:
            if not c["active"] or m < c["start"]:
                continue
            if rng_local.random() < churn_monthly:
                c["active"] = False
                continue
            g = rng_local.normal(cust_growth_mean, cust_growth_sd)
            c["demand"] *= max(0.95, (1.0 + g))
            total_demand += c["demand"]
        demand_clients_tpm[m] = total_demand

        dt = max(0.0, rng_local.normal(downtime_mean, downtime_sd))
        prod = P_tpm_base[m] * max(0.0, 1.0 - dt)
        prod_tpm[m] = prod

        Daddr_tpm = Daddr[m] / 12.0
        sales_tpm[m] = min(prod, total_demand, Daddr_tpm)

    return {
        "Capacity_tpy": Q_t,
        "Dm_tpy": Dm,
        "Daddr_tpy": Daddr,
        "DemandClients_tpy": demand_clients_tpm * 12.0,
        "Sales_tpy": sales_tpm * 12.0
    }

demo = simulate_one_path(growth_scenarios["central"], seed=123)


In [ ]:
plt.figure(figsize=(10,5))
plt.plot(t, demo["Capacity_tpy"], label="Capacité")
plt.plot(t, demo["Dm_tpy"], label="Marché")
plt.plot(t, demo["Daddr_tpy"], label="Max adressable")
plt.plot(t, demo["DemandClients_tpy"], label="Demande clients")
plt.plot(t, demo["Sales_tpy"], label="Ventes")
plt.xlabel("Temps (mois)")
plt.ylabel("t/an")
plt.title("Une trajectoire simulée — contraintes et ventes")
plt.legend()
plt.tight_layout()
plt.show()


## 7) Monte Carlo — quantiles P10/P50/P90

In [ ]:
def run_monte_carlo(n_sims=2000, scenario="central"):
    g = growth_scenarios[scenario]
    keys = ["Capacity_tpy","Dm_tpy","Daddr_tpy","DemandClients_tpy","Sales_tpy"]
    out = {k: np.zeros((n_sims, months+1), dtype=float) for k in keys}
    for i in range(n_sims):
        res = simulate_one_path(growth_annual=g)
        for k in keys:
            out[k][i,:] = res[k]
    return out

def quantiles(arr_2d):
    return {
        "P10": np.quantile(arr_2d, 0.10, axis=0),
        "P50": np.quantile(arr_2d, 0.50, axis=0),
        "P90": np.quantile(arr_2d, 0.90, axis=0),
        "Mean": np.mean(arr_2d, axis=0),
    }

mc = run_monte_carlo(n_sims=2000, scenario="central")
q_sales = quantiles(mc["Sales_tpy"])
q_cap   = quantiles(mc["Capacity_tpy"])
q_daddr = quantiles(mc["Daddr_tpy"])
q_dcli  = quantiles(mc["DemandClients_tpy"])


In [ ]:
plt.figure(figsize=(10,5))
plt.plot(t, q_cap["P50"], label="Capacité P50")
plt.plot(t, q_dcli["P50"], label="Demande clients P50")
plt.plot(t, q_daddr["P50"], label="Max adressable P50")
plt.plot(t, q_sales["P50"], label="Ventes P50")
plt.fill_between(t, q_sales["P10"], q_sales["P90"], alpha=0.2, label="Ventes P10–P90")
plt.axhline(11000, linestyle="--", linewidth=1)
plt.axhline(15000, linestyle="--", linewidth=1)
plt.xlabel("Temps (mois)")
plt.ylabel("t/an")
plt.title("Monte Carlo — scénario central")
plt.legend()
plt.tight_layout()
plt.show()


## 8) Seuils 11k / 15k (P50 ventes)

In [ ]:
def first_cross(series, threshold):
    idx = np.where(series >= threshold)[0]
    return int(idx[0]) if len(idx) else None

m_11 = first_cross(q_sales["P50"], 11000)
m_15 = first_cross(q_sales["P50"], 15000)

pd.DataFrame({
    "Seuil (t/an)": [11000, 15000],
    "Mois atteint (P50 ventes)": [m_11, m_15]
})


## 9) Points de calibration (comment rendre le modèle plus précis)
- Mettre à jour les hypothèses d'acquisition $\lambda(t)$ avec le pipeline réel.
- Mettre à jour la distribution volume/client (moyenne et dispersion) avec les premiers contrats.
- Ajuster $s(t)$ (adoption) avec les retours : durée d'homologation, multi-sourcing, inertie achats.
- Ajuster le ramp-up capacité avec TRS/arrêts réels.
